In [1]:
#We will attempt to perform content based filtering using 1. overview and 2. Keywords, genres, cast and crew
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

meta = pd.read_csv("movies_metadata_preprocessed.csv")
print(meta['overview'].head())
meta['overview'] = meta['overview'].fillna('')

count = TfidfVectorizer(stop_words='english')
matrix = count.fit_transform(meta['overview'])
cosine_sim = linear_kernel(matrix, matrix)

indices = pd.Series(meta.index, index=meta['title']).drop_duplicates()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object


In [2]:
print(cosine_sim)

[[1.         0.01501598 0.         ... 0.         0.00593763 0.        ]
 [0.01501598 1.         0.04686889 ... 0.         0.02199034 0.00927971]
 [0.         0.04686889 1.         ... 0.         0.01404055 0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.00593763 0.02199034 0.01404055 ... 0.         1.         0.        ]
 [0.         0.00927971 0.         ... 0.         0.         1.        ]]


In [3]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return meta['title'].iloc[movie_indices]


rec = get_recommendations("The Conjuring")
print(rec)

38020         The Conjuring 2
8087     The Boston Strangler
41622      Ghosts of Darkness
24885         The Borderlands
24041                   Ouija
39228       Chasing the Devil
19110          The Apparition
39115          Something Evil
16755               The Arbor
37937        8213: Gacy House
Name: title, dtype: object


In [4]:
meta = pd.read_csv("movies_metadata_preprocessed.csv")
credits = pd.read_csv('../credits.csv')
meta = meta.merge(credits, on = 'id')
meta = meta.dropna(subset = ['cast', 'crew'])
print(meta)

FileNotFoundError: File b'../credits.csv' does not exist

In [ ]:
from ast import literal_eval
print(type(meta['cast'][0]))
print(type(meta['crew'][0]))
print(type(meta['keywords'][0]))
print(type(meta['genres'][0]))
meta = meta.dropna(subset = ['keywords'])
meta = meta.dropna(subset = ['genres'])

# for i,row in meta.iterrows():
# #     meta['keywords'][i] = literal_eval(meta['keywords'][i])
#     print(meta['keywords'][i])
#     print(type(meta['keywords'][i]))  

In [ ]:
#Content based filtering using keywords, genres, cast and crew
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from ast import literal_eval


features = ['cast','crew','keywords', 'genres']

for feature in features:
    meta[feature] = meta[feature].apply(literal_eval)
    


In [ ]:
import numpy as np
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

def get_list(x):
    if isinstance(x, list):
        names = [i for i in x]
        if len(names) > 3:
            names = names[:3]
        return names
    
    return []
def get_list1(x):
    if isinstance(x, dict):
        names = [i['name'] for i in x]
        print(names)
        if len(names) > 3:
            names = names[:3]
        return names
    return []

def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

meta['director'] = meta['crew'].apply(get_director)

features = ['cast', 'keywords']

for feature in features:
    meta[feature] = meta[feature].apply(get_list)
    
meta['genres']=meta['genres'].apply(get_list1)
    
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    meta[feature] = meta[feature].apply(clean_data)
    print(feature)
    
def combine(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])
meta['combined'] = meta.apply(combine, axis=1)


count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(meta['combined'])

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

meta = meta.reset_index()
indices = pd.Series(meta.index, index=meta['title'])

In [ ]:
get_recommendations('The Conjuring', cosine_sim2)